In [18]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from scipy import signal
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import seaborn as sb
import keras.backend as K
from tensorflow.keras.utils import plot_model
import pickle

In [ ]:
#########################################################################
########################################################################
#######################################################################
######################################################################
#####################################################################
####################################################################

# Mobiact dataset

### Visualization Data

In [19]:
person_numlist = list(range(1, 68))
trials = list(range(1, 4))

In [20]:
# Loading ADLs (loading 5 times more adl data then fall, according to proportions)
# Taking 7 adls and 4 falls
# adl_types tells number of windows to extract from each data file

fall_types = ['FOL', 'FKL', 'BSC', 'SDL']
adl_types = {
    'STD':1,
    'WAL':1,
    'JOG':3,
    'JUM':3,
    'STU':6,
    'STN':6,
    'SIT':1
}

In [21]:
xtrain = []
ytrain = []

total_time = 0

for folder in os.listdir('MobiFall_CSV'):
    if folder not in adl_types : continue
    t1 = time.time()

    visualize = 1

    path = 'MobiFall_CSV/' + folder
    print('reading from ',path,end = '')
    for person in person_numlist:
        for trial in trials:
            try :
                data = pd.read_csv(path + '/' + folder + '_' + str(person) + '_' + str(trial) + '_' + 'FKL_gyro_9_3.csv')

                acc_x = np.array(data['acc_x']).reshape((len(data),1))
                acc_y = np.array(data['acc_y']).reshape((len(data),1))
                acc_z = np.array(data['acc_z']).reshape((len(data),1))
                gyro_x = np.array(data['gyro_x']).reshape((len(data),1))
                gyro_y = np.array(data['gyro_y']).reshape((len(data),1))
                gyro_z = np.array(data['gyro_z']).reshape((len(data),1))

                data = np.concatenate([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z],axis = -1)

                num_windows = (9+adl_types[folder])/adl_types[folder]
                for last_point in range(600,len(acc_x)+1,300):
                    num_windows -= 1
                    xtrain.append(data[last_point-600:last_point])
                    ytrain.append(0)
                    if num_windows == 0 : break

                if visualize == 1:
                    plt.figure(figsize = (12,2))
                    plt.plot(acc_x)
                    visualize = 0
                    print('===================================================================================================')
            except : continue

    t2 = time.time()
    total_time += t2 - t1
    print('Time taken :- ' , t2 - t1)
    plt.show()
print('Time taken == ',total_time)

Time taken ==  0


In [22]:
np.array(xtrain).shape

(0,)

In [28]:
# Visualizing acc_x of falls

print('FOL data :- ')
data = np.array(pd.read_csv('MobiFall_CSV/CSO_gyro_7_1.csv')['acc_x'])
plt.figure(figsize = (24,3))
plt.plot(data)
plt.show()

print('FKL data :- ')
data = np.array(pd.read_csv('MobiFall_CSV/BSC_gyro_1_1.csv')['gyro_Y'])
plt.figure(figsize = (24,3))
plt.plot(data)
plt.show()

print('BSC data :- ')
data = np.array(pd.read_csv('MobiFall_CSV/CSI_gyro_10_1.csv')['acc_Z'])
plt.figure(figsize = (24,3))
plt.plot(data)
plt.show()

print('SDL data :- ')
data = np.array(pd.read_csv('MobiFall_CSV/BSC_gyro_7_2.csv')['acc_x'])
plt.figure(figsize = (24,3))
plt.plot(data)
plt.show()

FOL data :- 


KeyError: 'acc_x'

In [30]:
# We extract using standard deviation

# Last visualization represents data before and after fall event extraction

total_time = 0

for folder in os.listdir('MobiFall_CSV'):
    if folder not in fall_types : continue
    t1 = time.time()

    visualize = 1

    path = 'MobiFall_CSV/' + folder
    print('reading from ',path,end = '')
    for person in person_numlist:
        for trial in trials:
            try :
                data = pd.read_csv(path + '/' + folder + '_' + str(person) + '_' + str(trial) + '_' + 'annotated.csv')

                acc_x = np.array(data['acc_x']).reshape((len(data),1))
                acc_y = np.array(data['acc_y']).reshape((len(data),1))
                acc_z = np.array(data['acc_z']).reshape((len(data),1))
                gyro_x = np.array(data['gyro_x']).reshape((len(data),1))
                gyro_y = np.array(data['gyro_y']).reshape((len(data),1))
                gyro_z = np.array(data['gyro_z']).reshape((len(data),1))

                data = np.concatenate([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z],axis = -1)

                acc_x_sd = ((acc_x - np.sum(acc_x)/len(acc_x))**2)
                # at = min(len(acc_x)-300,max(300,np.argmax(acc_x_sd)))
                at = np.argmax(acc_x_sd)

                if at-120 >= 0 and at+480 < len(acc_x):
                    xtrain.append(data[at-120:at+480])
                    ytrain.append(1)

                if at-240 >= 0 and at+360 < len(acc_x):
                    xtrain.append(data[at-240:at+360])
                    ytrain.append(1)

                if at-360 >= 0 and at+240 < len(acc_x):
                    xtrain.append(data[at-360:at+240])
                    ytrain.append(1)

                if at-480 >= 0 and at+120 < len(acc_x):
                    xtrain.append(data[at-480:at+120])
                    ytrain.append(1)

                if visualize == 1:
                    plt.figure(figsize = (12,2))
                    plt.plot(np.arange(at-300,at+300),acc_x[at-300:at+300],color = 'red')
                    plt.plot(np.arange(0,at-300),acc_x[0:at-300],color = 'blue')
                    plt.plot(np.arange(at+300,len(acc_x)),acc_x[at+300:],color = 'blue')
                    plt.show()
                    visualize = 0
                    print('===================================================================================================')
            except : continue

    t2 = time.time()
    total_time += t2 - t1
    print('Time taken :- ' , t2 - t1)
    plt.show()
print('Time taken == ',total_time)

xtrain = np.array(xtrain)
ytrain = np.array(ytrain)

Time taken ==  0


In [31]:
xtrain.shape,ytrain.shape

((0,), (0,))

In [33]:
xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,train_size = 0.8)
xtest,xval,ytest,yval = train_test_split(xtest,ytest,train_size = 0.5)

ValueError: With n_samples=0, test_size=None and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape,xval.shape,yval.shape

In [ ]:
# Min-max normalization xtrain ,xval and xtest data

for i in range(6):
    min_ = min([min(j) for j in xtrain[:,:,i]])
    max_ = max([max(j) for j in xtrain[:,:,i]])

    xtrain[:,:,i] = 2*(xtrain[:,:,i]-min_)/(max_-min_)-1

for i in range(6):
    min_ = min([min(j) for j in xtest[:,:,i]])
    max_ = max([max(j) for j in xtest[:,:,i]])

    xtest[:,:,i] = 2*(xtest[:,:,i]-min_)/(max_-min_)-1

for i in range(6):
    min_ = min([min(j) for j in xval[:,:,i]])
    max_ = max([max(j) for j in xval[:,:,i]])

    xval[:,:,i] = 2*(xval[:,:,i]-min_)/(max_-min_)-1

In [ ]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape,xval.shape,yval.shape

In [ ]:
# Graph of fall vs not fall

plt.figure(figsize = (24,8))
plt.subplot(1,3,1)
plt.bar(np.arange(0,2),[ytrain.shape[0]-sum(ytrain),sum(ytrain)])

plt.subplot(1,3,2)
plt.bar(np.arange(0,2),[ytest.shape[0]-sum(ytest),sum(ytest)])

plt.subplot(1,3,3)
plt.bar(np.arange(0,2),[yval.shape[0]-sum(yval),sum(yval)])
plt.show()

# Multiheaded CNN + BiLSTM

In [ ]:
# Multiheaded CNN

def build_cnn(input_layer):

    cnn = layers.Conv1D(9,7,padding="same")(input_layer)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    cnn = layers.Conv1D(18,5,padding="same")(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    cnn = layers.Conv1D(36,3,padding="same")(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    return cnn

In [ ]:
# Bi-LSTM Attentions

def build_bilstm(input_layer,last_sequences = False):

    lstm = layers.Bidirectional(layers.LSTM(18,return_sequences = True))(input_layer)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    lstm = layers.Bidirectional(layers.LSTM(36,return_sequences = True))(lstm)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    lstm = layers.Bidirectional(layers.LSTM(72,return_sequences = last_sequences))(lstm)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    return lstm

In [ ]:
# Dense layer

def build_dense(input_layer):

    dense = layers.Dense(72,name = 'dense_1')(input_layer)
    dense = layers.Dense(1,name = 'dense_3',activation = 'sigmoid')(dense)

    return dense

In [ ]:
def build_model():
    input_layer1 = keras.Input((xtrain.shape[1],3), name = 'Input1')
    input_layer2 = keras.Input((xtrain.shape[1],3), name = 'Input2')

    output1 = build_cnn(input_layer1)
    output2 = build_cnn(input_layer2)

    output = layers.concatenate([output1,output2])

    output = build_bilstm(output)

    output = build_dense(output)

    return keras.Model([input_layer1,input_layer2],output)

In [ ]:
plot_model(build_model(),show_shapes = True,dpi=40), build_model().summary()

In [ ]:
models1 = []

for BATCH_SIZE in [128,64,32]:
    for learning_rate in [0.0001]:
        print("\033[1m=====================================BATCH_SIZE=",BATCH_SIZE,"learning_rate=",learning_rate,"=====================================\033[0m")

        lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = learning_rate,
            decay_steps = int((xtrain.shape[0] + BATCH_SIZE)/BATCH_SIZE),
            decay_rate = 0.95
        )

        optimizer = keras.optimizers.Adam()
        optimizer.learning_rate = lr_scheduler

        model = build_model()

        plot_model(model,show_shapes = True,dpi=20)

        model.summary()

        model.compile(loss = 'binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
        model_history = model.fit(
            {'Input1': xtrain[:,:,0:3], 'Input2': xtrain[:,:,3:6]},
            ytrain,
            validation_data = (
                [xval[:,:,0:3],xval[:,:,3:6]],
                yval
            ),
            epochs = 50,
            batch_size = BATCH_SIZE,
        )

        # Testing accuracy

        plt.figure(figsize = (8,6))
        plt.plot(model_history.history['accuracy'], color="blue", alpha = 0.5)
        plt.plot(model_history.history['loss'], color="blue", alpha = 0.5)
        plt.plot(model_history.history['val_accuracy'], color="red", alpha = 0.5)
        plt.plot(model_history.history['val_loss'], color="red", alpha = 0.5)
        plt.show()

        print('test loss = ',model.evaluate([xtest[:,:,0:3],xtest[:,:,3:6]], ytest)[0])
        print('test accuracy = ',model.evaluate([xtest[:,:,0:3],xtest[:,:,3:6]], ytest)[1],'\n')

        ypred = model.predict([xtest[:,:,0:3],xtest[:,:,3:6]])

        test_cm = confusion_matrix(ytest,(ypred >= 0.5)*1)

        sb.set(rc = {'figure.figsize':(12.8,9.6)})
        sb.heatmap(test_cm,annot = True, cmap = 'YlOrRd')
        plt.show()

        print(sklearn.metrics.classification_report(ytest,(ypred >= 0.5)*1))

        models1.append(model)

# With Attentions

In [ ]:
class attention(layers.Layer):
    def __init__(self,return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="normal")

        super(attention,self).build(input_shape)

    def call(self,x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)

In [ ]:
# Multiheaded CNN

def build_cnn(input_layer):

    cnn = layers.Conv1D(9,7,padding="same")(input_layer)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    cnn = layers.Conv1D(18,5,padding="same")(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    cnn = layers.Conv1D(36,3,padding="same")(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(rate = 0.2)(cnn)

    return cnn

In [ ]:
# Bi-LSTM Attentions

def build_bilstm(input_layer,last_sequences = False):

    lstm = layers.Bidirectional(layers.LSTM(18,return_sequences = True))(input_layer)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    lstm = attention(True)(lstm)

    lstm = layers.Bidirectional(layers.LSTM(36,return_sequences = True))(lstm)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    lstm = attention(True)(lstm)

    lstm = layers.Bidirectional(layers.LSTM(72,return_sequences = last_sequences))(lstm)
    lstm = layers.LayerNormalization()(lstm)
    lstm = layers.Dropout(rate = 0.2)(lstm)

    return lstm

In [ ]:
# Dense layer

def build_dense(input_layer):

    dense = layers.Dense(72,name = 'dense_1')(input_layer)
    dense = layers.Dense(1,name = 'dense_3',activation = 'sigmoid')(dense)

    return dense

In [ ]:
def build_model():
    input_layer1 = keras.Input((xtrain.shape[1],3), name = 'Input1')
    input_layer2 = keras.Input((xtrain.shape[1],3), name = 'Input2')

    output1 = build_cnn(input_layer1)
    output2 = build_cnn(input_layer2)

    output = layers.concatenate([output1,output2])

    output = build_bilstm(output)

    output = build_dense(output)

    return keras.Model([input_layer1,input_layer2],output)

In [ ]:
plot_model(build_model(),show_shapes = True,dpi=40),build_model().summary()

In [ ]:
models2 = []

for BATCH_SIZE in [128,64,32]:
    for learning_rate in [0.0001]:
        print("\033[1m=====================================BATCH_SIZE=",BATCH_SIZE,"learning_rate=",learning_rate,"=====================================\033[0m")

        lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = learning_rate,
            decay_steps = int((xtrain.shape[0] + BATCH_SIZE)/BATCH_SIZE),
            decay_rate = 0.99
        )

        optimizer = keras.optimizers.Adam()
        optimizer.learning_rate = lr_scheduler

        model = build_model()

        plot_model(model,show_shapes = True,dpi=20)

        model.summary()

        model.compile(loss = 'binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
        model_history = model.fit(
            {'Input1': xtrain[:,:,0:3], 'Input2': xtrain[:,:,3:6]},
            ytrain,
            validation_data = (
                [xval[:,:,0:3],xval[:,:,3:6]],
                yval
            ),
            epochs = 50,
            batch_size = BATCH_SIZE,
        )

        # Testing accuracy

        plt.figure(figsize = (8,6))
        plt.plot(model_history.history['accuracy'], color="blue", alpha = 0.5)
        plt.plot(model_history.history['loss'], color="blue", alpha = 0.5)
        plt.plot(model_history.history['val_accuracy'], color="red", alpha = 0.5)
        plt.plot(model_history.history['val_loss'], color="red", alpha = 0.5)
        plt.show()

        print('test loss = ',model.evaluate([xtest[:,:,0:3],xtest[:,:,3:6]], ytest)[0])
        print('test accuracy = ',model.evaluate([xtest[:,:,0:3],xtest[:,:,3:6]], ytest)[1],'\n')

        ypred = model.predict([xtest[:,:,0:3],xtest[:,:,3:6]])

        test_cm = confusion_matrix(ytest,(ypred >= 0.5)*1)

        sb.set(rc = {'figure.figsize':(12.8,9.6)})
        sb.heatmap(test_cm,annot = True, cmap = 'YlOrRd')
        plt.plot()

        print(sklearn.metrics.classification_report(ytest,(ypred >= 0.5)*1))

        models2.append(model)

In [ ]:
import pickle
pickle.dump(models1, open('models1.sav', 'wb'))
pickle.dump(models2, open('models2.sav', 'wb'))
pickle.dump(xtrain, open('xtrain.sav', 'wb'))
pickle.dump(xtest, open('xtest.sav', 'wb'))
pickle.dump(xval, open('xval.sav', 'wb'))
pickle.dump(ytrain, open('ytrain.sav', 'wb'))
pickle.dump(ytest, open('ytest.sav', 'wb'))
pickle.dump(yval, open('yval.sav', 'wb'))